In [31]:
import pandas as pd
import numpy as np
import netCDF4 as nc

In [32]:
# 处理 CSV 文件的函数
def process_csv(file_path):
    # 读取文件并剔除异常数据
    df = pd.read_csv(file_path,encoding='gb2312')
    df['时间'] = pd.to_datetime(df['时间'])
    # 将日期时间转换为所需的数字格式  2024-04-20 05:31:00 -> 20240420053100
    df['时间'] = df['时间'].dt.strftime('%Y%m%d%H%M%S').astype('int64')
    index_temp = (df['温度'] < 40.0)&(df['温度'] > -2.5)
    df = df[index_temp]
    return df

In [33]:
file_path = r'C:\Users\lenovo\Desktop\海床基\SBB_BS_003_202303270530.csv'
# process_csv(file_path)
name = file_path.split('\\')[-1]

In [34]:
def nc_generate(df,new_path,file_path,longitude = 111.1234,latitude =23.1234, lon2 = 105.21,lat2 = 15.11,ReleaseTime = '2022-12-11 14:03:07 UTC',ReleaseLongitude = '111.8444°E',ReleaseLatitude = '17.4169°N',Comment = 'quality control data',information ='',Blanking = '0.5m',Cellsize = '8m',Cellnum = 10) :
    # SBB_Type:'BS' lon2:105.21 lat2:15.11 deploy_time:202303270530(整数就可以，字符串也可以)
    name = file_path.split('\\')[-1] # 'AWG_HYY_003_202303270530.csv'
    SBB_Type = name.split('_')[1]
    deviceID = name.split('_')[2]
    
    time = df['时间'][0]
    longitude =np.round(longitude,4); latitude =np.round(latitude,4); 
    seawater_pressure = df['压力']; seawater_pressure = np.round(seawater_pressure,3); 
    seawater_temperature = df['温度']; seawater_temperature = np.round(seawater_temperature,4); 
    seawater_salinity = df['盐度']; seawater_salinity = np.round(seawater_salinity,4); 

    import seawater
    seawater_density = seawater.dens(seawater_salinity, seawater_temperature, seawater_pressure)
    seawater_sound_velocity = seawater.svel(seawater_salinity, seawater_temperature, seawater_pressure)
    seawater_density = np.round(seawater_density,4); seawater_sound_velocity = np.round(seawater_sound_velocity,4); 

    subset = df.iloc[:, 7:]
    data = np.array(subset)
    data = data.reshape(-1,4)
    curr_x = data[:,0]; curr_x = np.round(curr_x,3); 
    curr_y = data[:,1]; curr_y = np.round(curr_y,3); 
    curr_z1 = data[:,2]; curr_z1 = np.round(curr_z1,3); 
    curr_z2 = data[:,3]; curr_z2 = np.round(curr_z2,3); 

    heading = df['艏向']; heading = np.round(heading,2); 
    roll = df['横摇']; roll = np.round(roll,2); 
    pitch = df['纵摇']; pitch = np.round(pitch,2); 
    

    num_row = df.shape[0] # 获取数据行数，为 node_dim准备
    # 创建nc文件
    format = 'NETCDF3_CLASSIC'
    last_time = str(time)[:12]
    filename = f'SBB_{SBB_Type}_{deviceID}_{lon2}E_{lat2}N_{last_time}_Q.nc'
    file  = nc.Dataset(new_path+'/'+f'{filename}', 'w',format= format)

    # 创建维度
    
    lon_dim = file.createDimension('num_longitude',1)
    lat_dim = file.createDimension('num_latitude',1)
    time_dim = file.createDimension('num_time',num_row) # num_row = 1
    node_dim = file.createDimension('num_node',num_row)
    cell_dim = file.createDimension('num_cell',Cellnum)

    # 创建变量
    time_var = file.createVariable('time','f8',('num_time',))
    lon_var = file.createVariable('longitude','f4',('num_longitude',))
    lat_var = file.createVariable('latitude','f4',('num_latitude',))
        # 'Latitude': 这是新创建的变量的名称
        # 'f4': 这指定了变量的数据类型。'f8'代表64位浮点数（双精度）
        # ('Latitude',):指定新变量所依赖的维度，('Latitude',)创建元组，而 ('Latitude')为一个字符串
    seawater_pressure_var = file.createVariable('seawater_pressure','f4',('num_node',))
    seawater_temperature_var = file.createVariable('seawater_temperature','f4',('num_node',))
    seawater_salinity_var = file.createVariable('seawater_salinity','f4',('num_node',))
    seawater_density_var = file.createVariable('seawater_density','f4',('num_node',))
    seawater_sound_velocity_var = file.createVariable('seawater_sound_velocity','f4',('num_node',))
    curr_x_var = file.createVariable('curr_x','f4',('num_cell',))
    curr_y_var = file.createVariable('curr_y','f4',('num_cell',))
    curr_z1_var = file.createVariable('curr_z1','f4',('num_cell',))
    curr_z2_var = file.createVariable('curr_z2','f4',('num_cell',))
    heading_var = file.createVariable('heading','f4',('num_node',))
    roll_var = file.createVariable('roll','f4',('num_node',))
    pitch_var = file.createVariable('pitch','f4',('num_node',))

    # 添加属性
    time_var.long_name="the UTC time observed by the seabed"
    lon_var.long_name = 'the longitude of the deployed seabed' 
    lon_var.units = "degrees_east" 
    lat_var.long_name = 'the latitude of the deployed seabed'
    lat_var.units = "degrees_north"
    seawater_pressure_var.long_name = "the seawater pressure observed by the seabed"
    seawater_pressure_var.units = "dbar"
    seawater_temperature_var.long_name = "the seawater temperature observed by the seabed"
    seawater_temperature_var.units = "Celsius" 
    seawater_salinity_var.long_name = "the seawater salinity observed by the seabed"
    seawater_salinity_var.units = "PSU" 
    seawater_density_var.long_name =  "the seawater density calculated from CTD"
    seawater_density_var.units = "kg/m3" 
    seawater_sound_velocity_var.long_name = "the seawater sound velocity calculated from CTD"
    seawater_sound_velocity_var.units = "m/s" 
    curr_x_var.long_name = 'the current velocity profile of seawater in the X-axis observed by the seabed'
    curr_x_var.units = "m/s" 
    curr_y_var.long_name = 'the current velocity profile of seawater in the Y-axis observed by the seabed'
    curr_y_var.units = "m/s"
    curr_z1_var.long_name = 'the current velocity profile of seawater in the Z1-axis observed by the seabed'
    curr_z1_var.units = "m/s"
    curr_z2_var.long_name = 'the current velocity profile of seawater in the Z2-axis observed by the seabed'
    curr_z2_var.units = " m/s"
    heading_var.long_name = 'the heading of ADCP integrated on the seabed'
    heading_var.units = "degree"
    roll_var.long_name = 'the roll of ADCP integrated on the seabed'
    roll_var.units = "degree"
    pitch_var.long_name = 'the pitch of ADCP integrated on the seabed'
    pitch_var.units = "degree"
    
    # 变量中写入数据
        # 必须time_var[:] = time_data，不可以time_var= time_data
        # 使用createVariable方法创建一个变量后，这个变量实际上是一个特殊的对象，它代表netCDF文件中的一个数据区域。
        # time_var = time_data会改变time_var的引用，而不是修改它所代表的数据区域的内容
        # 相反，time_var[:]是用来引用该变量整个数据区域的一个“切片”
        # time_var[:] = time_data 实际上是将time_data数组中的数据复制到time_var所代表的数据区域中
    time_var[:] = time
    lon_var[:] = longitude
    lat_var[:] = latitude
    seawater_pressure_var[:] = seawater_pressure
    seawater_temperature_var[:] = seawater_temperature
    seawater_salinity_var[:] = seawater_salinity
    seawater_density_var[:] = seawater_density
    seawater_sound_velocity_var[:] = seawater_sound_velocity
    curr_x_var[:] = curr_x
    curr_y_var[:] = curr_y
    curr_z1_var[:] = curr_z1
    curr_z2_var[:] = curr_z2
    heading_var[:] = heading
    roll_var[:] = roll
    pitch_var[:] = pitch

    # 全局属性(需要放到最后来，不然添加不上全局属性)
    file.SBB_SN = SBB_Type + '_' + deviceID
    file.SBB_ReleaseTime = ReleaseTime
    file.SBB_ReleaseLongitude = ReleaseLongitude
    file.SBB_ReleaseLatitude = ReleaseLatitude
    file.Sensor_accuracy = 'temperature,Seabird37 CTD,±0.002℃;conductivity,Seabird37 CTD,+0.0003S/m;pressure,Seabird37 CTD,±0.1% of full scale range;curr_speed,xxx,±0.01m/s'
    file.Comment = Comment
    file.Information = information
    file.Blanking = Blanking # 盲区厚度，如0.5m
    file.Cellsize = Cellsize # 层厚，如8m
    file.Cellnum = Cellnum # 层数，如10
    file.Manufacturer = '崂山实验室'
    file.Source = 'Wei Ma, TianJin University, wei.ma@tju.edu.cn'
    import datetime
    import pytz
    current_time = datetime.datetime.now() # 获取当前时间
    utc_time = current_time.astimezone(pytz.utc) # 将当前时间转换为 UTC 时间
    formatted_utc_time = utc_time.strftime("%Y-%m-%d %H:%M:%S %Z") # 格式化并显示 UTC 时间
    time_now = str(formatted_utc_time)[:19]
    file.History = f'Created by Python at {time_now} UTC'

    file.close() # 关闭文件
    nc_path = new_path+'/'+f'{filename}'
    return nc_path.replace('\\', '/')

In [35]:
file_path = r'C:\Users\lenovo\Desktop\SBB\SBB_BS_003_202303270530.csv'
df = process_csv(file_path=file_path)
# new_path = r'C:\Users\lenovo\Desktop\java\Glider05'
new_path = r'C:\Users\lenovo\Desktop\SBB' #'C:/Users/lenovo/Desktop/java/Glider05'  'C:/Users/lenovo/Desktop/任务帧nc数据'
longitude = 111.1234; latitude =23.1234
lon2 = 105.21 ;lat2 = 15.11 
nc_generate(df,new_path,file_path,longitude,latitude,lon2,lat2)

'C:/Users/lenovo/Desktop/SBB/SBB_BS_003_105.21E_15.11N_202404200531_Q.nc'

In [36]:
time = df['时间'][0]
last_time = str(time)[:12]
last_time
time

20240420053100